In [ ]:
"""
    Pappus' creativity is due to him (implicitly) distinguishing between numerical identity and conceptual identity
    in step 1 and applying this distinction to the triangle ABC to take advantage of Euclid I.4.
"""

!python --version

# install and import packages
!pip install rdflib

import glob
import json
import os
import rdflib
import requests
import typing

# !pwd

# PROOFS OF EUCLID I.5

In [ ]:
# import json
# import requests

class Row:
    """ Represents a single row of SPARQL query results. """
    def __init__(self, data):
        self.__dict__.update(data)
    """
    self.__dict__.update(data): 
    This line uses the update method 
    on the __dict__ attribute of the 
    object (self). The __dict__ attribute 
    is a special dictionary that stores 
    all the attributes (key-value pairs) 
    associated with the object. 
    By calling update on it with the 
    data dictionary, the __init__ method 
    effectively assigns all the key-value 
    pairs from the data dictionary 
    as attributes of the newly 
    created object.
    """

class SparqlQueryResults():
    """ Represents the results of a SPARQL query. """

    # initialize objects
    def __init__(self,
                sparql_query,
                rdfox_server:str = "http://localhost:12110",
                datastore:str = "/datastores/foom_pappus/sparql",
                format_response:str = "application/sparql-results+json"):
        self.sparql_query = sparql_query
        self.rdfox_server = rdfox_server
        self.datastore = datastore
        self.format_response = format_response
        self.response_text = self.query_ontology()
        self.response_dictionary = self.response_to_dictionary()
        self.rows = self.create_rows()
        self.index = 0  # For the iterator in the __next__ method

    # submit sparql query and retrieve results
    def query_ontology(self) -> str:
        """ Submit SPARQL query and retrieve results. """
        # define the headers of the request
        headers = {
            "Accept": self.format_response
        }
        
        # query the datastore at the rdfox server
        response = requests.get(f"{self.rdfox_server}{self.datastore}",
                                params={"query": self.sparql_query},
                                headers=headers)
        
        # check if the REST endpoint returns an unexpected status code
        response.raise_for_status()
        
        # if the REST endpoint returns an expected response, return the text of the response
        return response.text

    # transform the text response to the ontology query
    # into a python dictionary and
    # extract the relevant information about
    # headers and values
    def response_to_dictionary(self) -> dict:
        """ Transform the SPARQL query response into a dictionary. """
        
        # transform the text response to the ontology query
        # into a json object
        response_data = json.loads(self.response_text)

        # select the headers for the new dictionary of data
        headers:list = response_data["head"]["vars"]

        # transform the json response into a dictionary
        # with headers and values
        response_dictionary:dict = {header: [] for header in headers}
        for binding in response_data["results"]["bindings"]:
            for header in headers:
                response_dictionary[header].append(binding[header]["value"])
                
        return response_dictionary

    # create rows of results
    def create_rows(self) -> list:
        """ Create Row objects for each result. """

        # list to append rows
        rows:list = []

        # find header names and number of rows to add to the list of rows
        headers:list = list(self.response_dictionary.keys())
        number_of_rows= len(self.response_dictionary[headers[0]])

        # add rows to the list of rows
        for index in range(number_of_rows):
            row_data = {header:self.response_dictionary[header][index]
                       for header in headers}
            rows.append(Row(row_data))

        return rows

    # make the object iterable using
    # the __iter__ functions and
    # the __next_ function
    def __iter__(self):
        return self

    def __next__(self):
        """ Iterate through the rows of the results. """
        if self.index < len(self.rows):
            result = self.rows[self.index]
            self.index += 1
            return result
        else:
            self.index = 0  # Reset for future iteration
            raise StopIteration

In [ ]:
# %%time
################################
## FIND INFORMATION CONCERCING 
## THE CONCEPTUAL SPACE OF 
## A GIVEN PROOF STEP
################################

## select a proof step
# proof_step_iri:str = "<https://www.foom.com/pappus_proofAristotle#00000000000000000014>"
proof_step_analysis:dict = {"proof_step_iri": "<https://www.foom.com/pappus_proofAristotle#00000000000000000014>" }

# find label of proof step
def query_find_proof_step_label(proof_step_iri:str) -> str:
    sparql_query:str = f"""
        SELECT DISTINCT 
            ?proof_step_label
        WHERE {{
            {proof_step_iri} 
                rdfs:label ?proof_step_label .
        }}
    """
    return sparql_query

def find_proof_step_label(proof_step_analysis:dict,
                          results_query:str):
    for row in SparqlQueryResults(results_query):
        proof_step_analysis["proof_step_label"] = f"{row.proof_step_label}"
    return proof_step_analysis

proof_step_analysis:dict = find_proof_step_label(
                                                 proof_step_analysis,
                                                 query_find_proof_step_label(proof_step_analysis["proof_step_iri"])
                                                )

# find the proof to which the given proof step belongs
def query_find_proof(proof_step_iri:str) -> str:
    sparql_query:str = f"""
        SELECT DISTINCT 
            ?proof_iri
            ?proof_label
        WHERE {{
            {proof_step_iri} 
                <http://www.foom.com/core/inProof> ?proof_iri .
            ?proof_iri 
                rdfs:label ?proof_label .
        }}
    """
    return sparql_query

def find_proof(proof_step_analysis:dict,
               results_query:str):
    for row in SparqlQueryResults(results_query):
        proof_step_analysis["proof_iri"] = f"<{row.proof_iri}>"
        proof_step_analysis["proof_label"] = f"<{row.proof_label}>"
    return proof_step_analysis

proof_step_analysis:dict = find_proof(
                                      proof_step_analysis,
                                      query_find_proof(proof_step_analysis["proof_step_iri"])
                                     )

## find information concerning the proof:
# 1. goal
# 2. conceptual items connected to the goal
# 3. statement to prove
# 4. conceptual items connected to the statement to prove
# 5. source that contains the proof
# 6. author of the proof
# 7. remaining conceptual space of the proof

# goal
def query_find_goals_of_proof(proof_iri:str) -> str:
    sparql_query:str = f"""
        SELECT DISTINCT 
            ?goal_iri
            ?goal_label
        WHERE {{
            ?goal_iri
                <http://www.foom.com/core#00000000000000000013> {proof_iri} ; # is goal of proof_iri 
                rdfs:label ?goal_label .
        }}
    """
    return sparql_query

def find_goal_of_proof(proof_step_analysis:dict,
                       results_query:str):
    for row in SparqlQueryResults(results_query):
        proof_step_analysis["proof_goal_iri"] = f"<{row.goal_iri}>"
        proof_step_analysis["proof_goal_label"] = f"<{row.goal_label}>"
    return proof_step_analysis

proof_step_analysis:dict = find_goal_of_proof(
                                              proof_step_analysis,
                                              query_find_goals_of_proof(proof_step_analysis["proof_iri"])
                                             )

## find preceding proofs steps
def query_find_antecedent_proof_steps(proof_step_iri:str,
                                      proof_iri:str) -> str:
    sparql_query:str = f"""
        SELECT 
            ?antecedent_iri
        WHERE {{
            {proof_step_iri}
                <http://www.foom.com/core#00000000000000000186> ?antecedent_iri .
            ?antecedent_iri <http://www.foom.com/core/inProof> {proof_iri} .
        }}
    """
    return sparql_query

def find_antecedent_proof_steps(proof_step_analysis:dict,
                                results_query:str):
    proof_step_analysis["antencedent_proof_steps"] = [row.antecedent_iri for row in SparqlQueryResults(results_query)]
    return proof_step_analysis

proof_step_analysis:dict = find_antecedent_proof_steps(proof_step_analysis,
                                                       query_find_antecedent_proof_steps(
                                                           proof_step_analysis["proof_step_iri"],proof_step_analysis["proof_iri"]
    )
)

## find the conceptual space of the preceding proof steps

## find analogous proofs 

## find analogous proof parts

## find other proofs or proof parts connected to the given proof step or proof


proof_step_analysis

In [ ]:
#### SPARQL QUERIES
## 0. find all proofs
def query_find_proofs() -> str:
    sparql_query:str = """
        SELECT DISTINCT 
            ?proof_iri
            ?proof_label
        WHERE {
            ?proof_iri 
                a <http://www.foom.com/core/Proof> ;
                rdfs:label ?proof_label .
        }
    """
    return sparql_query

# run the sparql query to find proofs 
# and put the results in a dictionary
def find_proofs(sparql_query:str = query_find_proofs()):
    query_results_proofs = SparqlQueryResults(sparql_query)
    proofs:dict = {}
    for row in query_results_proofs:
        proofs[str(row.proof_iri)] = {"label": str(row.proof_label)}
    return proofs

## 1. goals of proofs and statements to prove
# find goals of proofs
def query_find_goals_of_proofs() -> str:
    sparql_query:str = """
        SELECT DISTINCT 
            ?proof_iri
            ?goal_iri
            ?goal_label
        WHERE {
            ?proof_iri 
                a <http://www.foom.com/core/Proof> .
            ?goal_iri
                <http://www.foom.com/core/inProof> ?proof_iri ; # is in proof_iri
                <http://www.foom.com/core#00000000000000000013> ?proof_iri ; # is goal of proof_iri 
                rdfs:label ?goal_label .
        }
    """
    return sparql_query

# find statements to prove
def query_find_statements_to_prove() -> str:
    sparql_query:str = """
        SELECT DISTINCT 
            ?proof_iri
            ?claim_iri
            ?claim_label
        WHERE {
            ?proof_iri 
                a <http://www.foom.com/core/Proof> ;
                <http://www.foom.com/mathematical_concepts#00000000000000000250> ?claim_iri . # proves claim iri
            ?claim_iri rdfs:label ?claim_label .
        }
    """
    return sparql_query

# run queries and put the results in the proofs dictionary
def find_statement_to_prove(proofs:dict) -> dict:
    goals_of_proofs = SparqlQueryResults(query_find_goals_of_proofs())
    statements_of_proofs = SparqlQueryResults(query_find_statements_to_prove())
    for row in goals_of_proofs:
        proofs[str(row.proof_iri)]["goal"] = {"iri": str(row.goal_iri),
                                             "iri_label": str(row.goal_label)}
    for row in statements_of_proofs:
        proofs[str(row.proof_iri)]["statement_to_prove"] = {"iri": str(row.claim_iri),
                                                           "iri_label": str(row.claim_label)}
    return proofs


## 2. conceptual items connected to the goal and the statement of the proofs
# give an item, find its reification values
def query_find_reification_values(ontology_item_iri:str) -> str:
    sparql_query:str = f"""
        SELECT DISTINCT 
            ?subject_iri ?subject_label
            ?predicate_iri ?predicate_label
            ?object_iri ?object_label
        WHERE {{
            <{ontology_item_iri}>
                <http://www.foom.com/core#00000000000000000087> ?subject_iri ; # find subject
                <http://www.foom.com/core#00000000000000000089> ?predicate_iri ; # find predicate
                <http://www.foom.com/core#00000000000000000088> ?object_iri . # find object
            ?subject_iri rdfs:label ?subject_label .
            ?predicate_iri rdfs:label ?predicate_label .
            ?object_iri rdfs:label ?object_label .
        }}
    """
    return sparql_query

def find_reification_values(proofs:dict) -> dict:
    for proof_iri in proofs:
        goal_iri = proofs[proof_iri]["goal"]["iri"]
        reification_values = SparqlQueryResults(query_find_reification_values(goal_iri))
        for row in reification_values:
            proofs[proof_iri]["goal"]["reification_values"] = {
                "subject_iri": str(row.subject_iri),
                "subject_label": str(row.subject_label),
                "predicate_iri": str(row.predicate_iri),
                "predicate_label": str(row.predicate_label),
                "object_iri": str(row.object_iri),
                "object_label": str(row.object_label)}
    return proofs

In [ ]:
## 0. find all proofs
proofs:dict = find_proofs()

## 1. goals of proofs and statements to prove
proofs:dict = find_statement_to_prove(proofs)

## 2. conceptual items connected to the goal and the statement of the proofs
proofs:dict = find_reification_values(proofs)

# http://www.foom.com/mathematical_concepts#00000000000000000153
def query_find_conceptual_relations_values_of_goal(ontology_item_iri:str) -> str:
    sparql_query:str = f"""
        SELECT DISTINCT 
            ?conceptual_relation_iri ?conceptual_relation_label
            ?conceptually_related_item_iri ?conceptually_related_item_label
        WHERE {{
            ?conceptual_relation_iri
                rdfs:subPropertyOf+ <http://www.foom.com/mathematical_concepts#00000000000000000153> 
                rdfs:label ?conceptual_relation_label .
            <{ontology_item_iri}> 
                ?conceptual_relation_iri ?conceptually_related_item_iri .
            ?conceptually_related_item_iri rdfs:label ?conceptually_related_item_label .
        }}
    """
    return sparql_query

def find_conceptua_relations_values_of_goal(proofs:dict) -> dict:
    for proof_iri in proofs:
        for key in proofs[proof_iri]["goal"]["reification_values"]:
            if key.endswith("iri"):
                print(key)
                conceptual_relations_values = SparqlQueryResults(query_find_conceptual_relations_values_of_goal([proof_iri]["goal"]["reification_values"][key]))
    return proofs

proof = find_conceptua_relations_values_of_goal(proofs)
# 3. apply heuristics
# 4. compare step with results of heuristics
# 5. repeat for each step

proofs 

In [ ]:
results = SparqlQueryResults(query_find_conceptaul_relations_values("https://www.foom.com/pappus_proofPappus#00000000000000000001"))
for row in results:
    print(row.conceptual_relation_iri)

In [ ]:
#### SPARQL QUERIES

# find all proofs
def sparql_query_find_proofs():
    sparql_query:str = """
        SELECT DISTINCT 
            ?proof_iri
            ?proof_label
        WHERE {
            ?proof_iri 
                a <http://www.foom.com/core/Proof> ;
                rdfs:label ?proof_label .
        }
    """
    return sparql_query

# for every proof, find all the steps in that proof
def sparql_query_find_steps_in_proof(given_proof_iri:str):
    sparql_query:str = f"""
    SELECT DISTINCT
        ?proof_iri
        ?proof_step_iri
        ?proof_step_label
        ?antecendent_proof_step_iri
    WHERE {{
        ?proof_iri 
            a <http://www.foom.com/core/Proof> .
        ?proof_step_iri
            a <http://www.foom.com/core/Proof_step> ;
            rdfs:label ?proof_step_label ;
            <http://www.foom.com/core/inProof> {given_proof_iri} .
        OPTIONAL {{
            ?proof_step_iri <http://www.foom.com/core/hasAntecedent> ?antecendent_proof_step_iri .
        }}
    }}
    """
    return sparql_query

# find all the individuals and properties used in the proof steps of a given proof
# group by proof step
def sparql_query_find_individuals_and_properties_of_proof_steps_of_given_proof(given_proof_iri:str):
    sparql_query:str = f"""
    SELECT DISTINCT
        ?proof_iri
        ?proof_step_iri
    WHERE {{
        ?proof_iri 
            a <http://www.foom.com/core/Proof> .
        ?proof_step_iri
            a <http://www.foom.com/core/Proof_step> ;
            <http://www.foom.com/core/inProof> {given_proof_iri} .
        
    }}
    """
    return sparql_query

# find exactly all the properties used in some proof step
def sparql_query_find_properties_proof_steps():
    sparql_query:str = """
        SELECT DISTINCT 
            ?p 
            ?p_label
            ?p_type
        WHERE {
            ?proof_step_iri 
                a <http://www.foom.com/core/Proof_step> ;
                ?p ?o .
            ?p 
                rdfs:label ?p_label ;
                a ?p_type .
        }
    """
    return sparql_query
    

# for every step, find the triple attached to it and the items to which the step directly refers

